### Emulating a controller with an LSTM Network

The purpose of this exercise is to automate a temperature control process with an LSTM network. The LSTM network is trained from an MPC (Model Predictive Control). LSTM (Long Short Term Memory) networks are a special type of RNN (Recurrent Neural Network) that is structured to remember and predict based on long-term dependencies that are trained with time-series data. An LSTM repeating module has four interacting components.

<img align=left width=400px src='https://apmonitor.com/pds/uploads/Main/lstm_automation.png'>

The LSTM is trained (parameters adjusted) with an input window of prior data and minimized difference between the predicted and next measured value. Sequential methods predict just one next value based on the window of prior data. In this case, the error between the set point and measured value is the feature and the heater value is the output label.

See [Automation with LSTM Networks](https://apmonitor.com/pds/index.php/Main/LSTMAutomation) for additional details.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import time
from tqdm import tqdm # Progress bar

# For scaling, feature selection
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split 

# For LSTM model
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from keras.callbacks import EarlyStopping
from tqdm.keras import TqdmCallback
from keras.models import load_model

# For Gekko and TCLab
import tclab
from gekko import GEKKO

### Use [TCLab](https://apmonitor.com/heat.htm) Microcontroller or Emulator (Digital Twin)

Change `tclab_hardware` to `True` if TCLab device is connected.

<img width=550px align=left src='https://apmonitor.com/pdc/uploads/Main/tclab_connect.png'>

In [ ]:
tclab_hardware = False # True
if tclab_hardware:
    mlab = tclab.TCLab      # Physical hardware
else:
    mlab = tclab.TCLabModel # Emulator

### PID controller 

See [LSTM Network Replaces PID](https://github.com/APMonitor/pds/blob/main/LSTM_Automation.ipynb) for code with a PID controller instead of a Model Predictive Controller (MPC).

### MPC (Model Predictive Control)

See the last example for [MPC with TCLab](https://apmonitor.com/pds/notebooks/12_time_series.html) or the [Linear MPC with TCLab](https://apmonitor.com/do/index.php/Main/TCLabF). This controller is implemented instead of PID control. The steps to create an MPC application are:

- identify model
- initialize controller
- create MPC function to return Q1

**Identify Model**

In [ ]:
#########################################################
# Initialize Model
#########################################################
# load data (20 min, dt=2 sec) and parse into columns
url = 'http://apmonitor.com/do/uploads/Main/tclab_2sec.txt'
data = pd.read_csv(url)
t = data['Time']
u = data[['H1','H2']]
y = data[['T1','T2']]

# generate time-series model
m = GEKKO(remote=False)

##################################################################
# system identification
na = 2 # output coefficients
nb = 2 # input coefficients
print('Identify model')
yp,p,K = m.sysid(t,u,y,na,nb,objf=10000,scale=False,diaglevel=0)

##################################################################
# plot sysid results
plt.figure()
plt.subplot(2,1,1)
plt.plot(t,u)
plt.legend([r'$Q_1$',r'$Q_2$'])
plt.ylabel('MVs')
plt.subplot(2,1,2)
plt.plot(t,y)
plt.plot(t,yp)
plt.legend([r'$T_{1meas}$',r'$T_{2meas}$',\
            r'$T_{1pred}$',r'$T_{2pred}$'])
plt.ylabel('CVs')
plt.xlabel('Time')
plt.savefig('sysid.png')
plt.show()

**Initialize Controller**

Create 2x2 (2 Heaters, 2 Temperature Sensors) MPC but only use 1x1 (`Q1`, `T1`) for this application. The LSTM learning can be extended to the 2x2 MPC (MVs: `Q1` and `Q2`, CVs: `T1` and `T2`).

In [ ]:
##################################################################
# create control ARX model
m = GEKKO(remote=False)
m.y = m.Array(m.CV,2)
m.u = m.Array(m.MV,2)
m.arx(p,m.y,m.u)

# rename CVs
m.TC1 = m.y[0]
m.TC2 = m.y[1]

# rename MVs
m.Q1 = m.u[0]
m.Q2 = m.u[1]

# steady state initialization
m.options.IMODE = 1
m.solve(disp=False)

# set up MPC
m.options.IMODE   = 6 # MPC
m.options.CV_TYPE = 1 # Objective type
m.options.NODES   = 2 # Collocation nodes
m.options.SOLVER  = 1 # APOPT
m.time=np.linspace(0,120,61)

# Manipulated variables
m.Q1.STATUS = 1  # manipulated
m.Q1.FSTATUS = 0 # not measured
m.Q1.DMAX = 50.0
m.Q1.DCOST = 0.1
m.Q1.UPPER = 100.0
m.Q1.LOWER = 0.0

m.Q2.STATUS = 0  # manipulated, turn off Q2
m.Q2.FSTATUS = 1 # use measured value
m.Q2.DMAX = 50.0
m.Q2.DCOST = 0.1
m.Q2.UPPER = 100.0
m.Q2.LOWER = 0.0
m.Q2.MEAS = 0    # set Q2=0

# Controlled variables
m.TC1.STATUS = 1     # drive to set point
m.TC1.FSTATUS = 1    # receive measurement
m.TC1.TAU = 20       # response speed (time constant)
m.TC1.TR_INIT = 2    # reference trajectory
m.TC1.TR_OPEN = 0

m.TC2.STATUS = 0     # drive to set point
m.TC2.FSTATUS = 1    # receive measurement
m.TC2.TAU = 20       # response speed (time constant)
m.TC2.TR_INIT = 2    # dead-band
m.TC2.TR_OPEN = 1

In [ ]:
def mpc(m,T1,T1sp,T2,T2sp):    
    # Insert measurements
    m.TC1.MEAS = T1
    m.TC2.MEAS = T2

    # Adjust setpoints
    db1 = 1.0 # dead-band
    m.TC1.SPHI = T1sp + db1
    m.TC1.SPLO = T1sp - db1

    db2 = 0.2
    m.TC2.SPHI = T2sp + db2
    m.TC2.SPLO = T2sp - db2
    
    # Adjust heaters with MPC
    m.solve(disp=False) 

    if m.options.APPSTATUS == 1:
        # Retrieve new values
        Q1  = m.Q1.NEWVAL
        Q2  = m.Q2.NEWVAL
    else:
        # Solution failed
        Q1  = 0.0
        Q2  = 0.0    
    return [Q1,Q2]

### Generate data for training LSTM

In [ ]:
##### Set up run parameters #####
# Run time in minutes
run_time = 90.0 # 90.0

# Number of cycles
loops = int(30.0*run_time+1)

# arrays for storing data
T1 = np.zeros(loops) # measured T (degC)
T2 = np.zeros(loops) # measured T (degC)
Q1 = np.zeros(loops) # Heater values
Q2 = np.zeros(loops) # Heater values
tm = np.linspace(0,2*(loops-1),loops) # Time

# Temperature set point (degC)
with mlab() as lab:
    Tsp1 = np.ones(loops) * lab.T1
    Tsp2 = np.ones(loops) * lab.T2

# vary temperature setpoint
end = 2 # leave first couple cycles of temp set point as room temp
while end <= loops:
    start = end
    # keep new temp set point value for anywhere from 4 to 10 min
    end += random.randint(240,600) 
    Tsp1[start:end] = random.randint(30,70)
    
if tclab_hardware:
    # print every cycle with hardware
    p1 = 10; p2 = 1
else:
    # print 10x less with emulator
    p1 = 100; p2 = 10   

# Plot
plt.plot(tm,Tsp1,'b.-')
plt.xlabel('Time',size=14)
plt.ylabel(r'Temp SP ($^oC$)',size=14)
plt.xticks(size=12)
plt.yticks(size=12)
plt.savefig('SP_profile.png');

### Data Collection

In [ ]:
# Data collection
with mlab() as lab:
    # Find current T1, T2
    print('Temperature 1: {0:0.2f} °C'.format(lab.T1))
    print('Temperature 2: {0:0.2f} °C'.format(lab.T2))
    
    t1 = time.time()
    for i in range(loops):
        t2 = time.time()
        
        if tclab_hardware:
            if tm[i]>=(t2-t1):
                time.sleep(max(0,min(2,tm[i]-(t2-t1))))
        else:
            for t in tclab.clock(2, 2):
                continue
                
        # Read temperatures in Celcius 
        T1[i] = lab.T1; T2[i] = lab.T2

        # Calculate MPC output every 2 sec
        [Q1[i],Q2[i]] = mpc(m,T1[i],Tsp1[i],T2[i],Tsp2[i])
        # Write heater output (0-100)
        lab.Q1(Q1[i])
        lab.Q2(Q2[i])
         
        if i%p1==0:            
            print('  Time_____Q1___Tsp1_____T1______Q2____Tsp2_____T2')
        if i%p2==0:
            print(('{:6.1f} {:6.2f} {:6.2f} {:6.2f}  {:6.2f}  {:6.2f} {:6.2f}').format( \
                      tm[i],Q1[i],Tsp1[i],T1[i],Q2[i],Tsp2[i],T2[i]))

In [ ]:
# Save csv file
df = pd.DataFrame()
df['Q1'] = Q1
df['T1'] = T1
df['Tsp'] = Tsp1
df.to_csv('MPC_train_data.csv',index=False)

# Plot
df.plot()
plt.savefig('MPC_train.png');

### Feature engineering: create SP error feature and analyze feature importance

In [ ]:
# Create new feature: setpoint error
df['err'] = df['Tsp'] - df['T1']

# Load possible features
X = df[['T1','Tsp','err']]
y = np.ravel(df[['Q1']])

# SelectKBest feature selection
bestfeatures = SelectKBest(score_func=f_regression, k='all')
fit = bestfeatures.fit(X,y)
plt.bar(x=X.columns,height=fit.scores_);

### Feature selection, scaling, and formatting data to LSTM input format

In [ ]:
X = df[['Tsp','err']].values
y = df[['Q1']].values

# Scale data
s_x = MinMaxScaler()
Xs = s_x.fit_transform(X)

s_y = MinMaxScaler()
ys = s_y.fit_transform(y)

# Each input uses last 'window' number of Tsp and err to predict the next Q1
window = 15
X_lstm = []
y_lstm = []
for i in range(window,len(df)):
    X_lstm.append(Xs[i-window:i])
    y_lstm.append(ys[i])

# Reshape data to format accepted by LSTM
X_lstm, y_lstm = np.array(X_lstm), np.array(y_lstm)

# Split into train and test 
Xtrain, Xtest, ytrain, ytest = train_test_split(X_lstm,y_lstm,test_size=0.2,shuffle=False)

### Keras LSTM 

In [ ]:
# Keras LSTM model
model = Sequential()

# First layer specifies input_shape and returns sequences
model.add(LSTM(units=100, return_sequences=True, 
               input_shape=(Xtrain.shape[1],Xtrain.shape[2])))
# Dropout layer to prevent overfitting
model.add(Dropout(rate=0.1))

# Last layer doesn't return sequences (middle layers should return sequences)
model.add(LSTM(units=100))
model.add(Dropout(rate=0.1))

# Dense layer to return prediction
model.add(Dense(1))

# Compile model; adam optimizer, mse loss
model.compile(optimizer='adam', loss='mean_squared_error')

es = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=25)

result = model.fit(Xtrain, ytrain, verbose=0, validation_split=0.2,
                   callbacks = [TqdmCallback(verbose=1)],#es
                   batch_size=100,
                   epochs=300)

# Plot loss and save model
epochs = es.stopped_epoch
plt.semilogy(result.history['loss'],label='loss')
plt.semilogy(result.history['val_loss'],label='val_loss')
plt.legend()

model.save('lstm_control.h5')

### Preliminary model performance assessment

In [ ]:
# Predict using LSTM
yp_s = model.predict(Xtest)

# Unscale data
Xtest_us = s_x.inverse_transform(Xtest[:,-1,:])
ytest_us = s_y.inverse_transform(ytest)
yp = s_y.inverse_transform(yp_s)

# Derive Tsp (sp) and T1 (pv) from X data
sp = Xtest_us[:,0]
pv = Xtest_us[:,0] + Xtest_us[:,1]

# Plot SP, MPC response, and LSTM response
plt.plot(sp,'k-',label='$SP$ $(^oC)$')
plt.plot(pv,'r-',label='$T_1$ $(^oC)$')
plt.plot(ytest_us,'b-',label='$Q_{MPC}$ (%)')
plt.plot(yp,'g-',label='$Q_{LSTM}$ (%)')
plt.legend(fontsize=12,loc='lower right')
plt.xlabel('Time',size=14)
plt.ylabel('Value',size=14)
plt.xticks(size=12)
plt.yticks(size=12);

### Generate SP data for test

In [ ]:
# Run time in minutes
run_time = 45.0

# Number of cycles
loops = int(30.0*run_time)

# arrays for storing data
T1 = np.zeros(loops) # measured T (degC)
T2 = np.zeros(loops)
Q1mpc = np.zeros(loops) # Heater values for MPC controller
Q2mpc = np.zeros(loops)
Qlstm = np.zeros(loops) # Heater values for LSTM controller
tm = np.linspace(0,2*(loops-1),loops) # Time

# Temperature set point (degC)
with mlab() as lab:
    Tsp1 = np.ones(loops) * lab.T1
    Tsp2 = np.ones(loops) * lab.T2

# vary temperature setpoint
end = window + 5 # leave 1st window + 10 seconds of temp set point as room temp
while end <= loops: 
    start = end
    # keep new temp set point value for anywhere from 4 to 10 min
    end += random.randint(240,600) 
    Tsp1[start:end] = random.randint(30,70)
    
# leave last 120 seconds as room temp
Tsp1[-120:] = Tsp1[0]
plt.plot(Tsp1)
plt.show()

### Part A: Run the controller with the PID, but also output the LSTM for comparison

In [ ]:
# LSTM Controller
def lstm(T1_m, Tsp_m):
    # Calculate error (necessary feature for LSTM input)
    err = Tsp_m - T1_m
    
    # Format data for LSTM input
    X = np.vstack((Tsp_m,err)).T
    Xs = s_x.transform(X)
    Xs = np.reshape(Xs, (1, Xs.shape[0], Xs.shape[1]))
    
    # Predict Q for controller and unscale
    Q1c_s = model.predict(Xs)
    Q1c = s_y.inverse_transform(Q1c_s)[0][0]
    
    # Ensure Q1c is between 0 and 100
    Q1c = np.clip(Q1c,0.0,100.0)
    return Q1c

In [ ]:
# Run test
with mlab() as lab:
    # Find current T1, T2
    print('Temperature 1: {0:0.2f} °C'.format(lab.T1))
    print('Temperature 2: {0:0.2f} °C'.format(lab.T2))
    
    t1 = time.time()
    for i in range(loops):
        t2 = time.time()
        
        if tclab_hardware:
            if tm[i]>=(t2-t1):
                time.sleep(max(0,min(2,tm[i]-(t2-t1))))
        else:
            for t in tclab.clock(2, 2):
                continue

        # Read temperatures in Celcius 
        T1[i] = lab.T1; T2[i] = lab.T2

        # Calculate MPC output every 2 sec
        [Q1mpc[i],Q2mpc[i]] = mpc(m,T1[i],Tsp1[i],T2[i],Tsp2[i])
        # Write heater output (0-100)
        lab.Q1(Q1mpc[i])
        lab.Q2(Q2mpc[i])
         
        if i%p1==0:            
            print('  Time_____Q1___Tsp1_____T1')
        if i%p2==0:
            print(('{:6.1f} {:6.2f} {:6.2f} {:6.2f}').format( \
                      tm[i],Q1mpc[i],Tsp1[i],T1[i]))
        
        # Run LSTM model to get Q1 value for control
        if i >= window:
            # Load data for model
            T1_m = T1[i-window:i]
            Tsp_m = Tsp1[i-window:i]
            # Predict and store LSTM value for comparison
            Qlstm[i] = lstm(T1_m,Tsp_m)
            
        prev_time = t

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(Tsp1[:i],'k-',label='SP $(^oC)$')
plt.plot(T1[:i],'r-',label='$T_1$ $(^oC)$')
plt.plot(Q1mpc[:i],'b-',label='$Q_{MPC}$ (%)')
plt.plot(Qlstm[:i],'g-',label='$Q_{LSTM}$ (%)')
plt.legend(loc='upper right',fontsize=14)
plt.ylim((0,100))
plt.xlabel('Time (s)',size=14)
plt.xticks(size=12)
plt.yticks(size=12)
plt.plot()

### Part B: Run the controller with just the LSTM

In [ ]:
# Run test
with mlab() as lab:
    # Find current T1, T2
    print('Temperature 1: {0:0.2f} °C'.format(lab.T1))
    print('Temperature 2: {0:0.2f} °C'.format(lab.T2))
    
    t1 = time.time()
    for i in range(loops):
        t2 = time.time()
        
        if tclab_hardware:
            if tm[i]>=(t2-t1):
                time.sleep(max(0,min(2,tm[i]-(t2-t1))))
        else:
            for t in tclab.clock(2, 2):
                continue

        # Read temperatures in Celcius 
        T1[i] = lab.T1; T2[i] = lab.T2

        # Run LSTM model to get Q1 value for control
        if i >= window:
            # Load data for model
            T1_m = T1[i-window:i]
            Tsp_m = Tsp[i-window:i]
            # Predict and store LSTM value for comparison
            Qlstm[i] = lstm(T1_m,Tsp_m)

        # Write heater output (0-100)
        lab.Q1(Qlstm[i])

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(Tsp[:i],'k-',label='SP $(^oC)$')
plt.plot(T1[:i],'r-',label='$T_1$ $(^oC)$')
plt.plot(Qlstm[:i],'g-',label='$Q_{LSTM}$ (%)')
plt.legend(fontsize=14)
plt.ylim((0,100))
plt.xlabel('Time (s)',size=14)
plt.xticks(size=12)
plt.yticks(size=12)
plt.grid()
plt.show()

### Part C

Use the LSTM controller to control the [TCLab microcontroller](https://apmonitor.com/heat.htm) or [Emulated TCLab](https://tclab.readthedocs.io/en/latest/notebooks/04_Emulation_of_TCLab_for_Offline_Use.html) from a separate program. Demonstrate that you can [package and deploy the LSTM model](https://apmonitor.com/pds/index.php/Main/DeployMachineLearning) so that it could be used on a different computer than the training computer. The application does not necessarily need to run on a separate computer, but show that it is a separate program and independent of this training notebook.

<img align=left width=150px src='https://apmonitor.com/pds/uploads/Main/deploy_machine_learning.png'>

<img align=left width=400px src='https://apmonitor.com/pdc/uploads/Main/tclab_connect.png'>

In [ ]:
import pickle
# export model name and scaling
z = ['lstm_control.h5',s_x,s_y,window]
pickle.dump(z,open('lstm_control.pkl','wb'))